# Notes

1. Zillow blocks repeated pings/downloads. you need to use a proxy. I couldn't figure it out solely in python, but found this website: https://app.zenrows.com/builder
    - I had to refresh a few times (I think the proxy gets noticed and then it issues you another) but eventually downloaded all pages in the target zip codes to .json 
    - It might be easy to use their code/service to this all 100% programmatically, but I couldn't figure it out. 
    
2. This is the zillow data as of Dec 10th for the zip codes displayed on the map in the #data-team thread. 

In [ ]:
## importing downloaded jsons to make single DF. 
import glob, os, json, warnings
import pandas as pd
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)

data_dir = './Zillow_Data/'
json_paths = glob.glob(os.path.join(data_dir,'*.json'))

temp_dfs = []
for j in json_paths:
    with open(j) as zil_json:
        jdata = pd.json_normalize(json.loads(zil_json.read()))
#         print(jdata.shape)
#         jdata.head()
    temp_dfs.append(jdata)
    
full_df = pd.concat(temp_dfs)
# print(full_df.shape)

## clean up df
col_of_interest = ['statusText','price',
                   'address','addressZipcode',
                   'beds','baths','area',
                   'rentZestimate','variableData.text',
                   'units',
                  ]
full_df.drop(columns=[col for col in full_df if col not in col_of_interest], inplace=True)
full_df.reset_index(drop=True, inplace=True)

all_cols = list(full_df)
full_df[all_cols] = full_df[all_cols].astype('string')
full_df['price'] = full_df['price'].str.replace('+','')
full_df['price'] = full_df['price'].str.replace('$','')
full_df['price'] = full_df['price'].str.replace(',','')
full_df['price'] = full_df['price'].str.replace('/mo','')
# print('pre shape:')
# print(full_df.shape)

## clean up missing rent/multiple units
df4append = pd.DataFrame(columns = full_df.columns)
for index, row in full_df.iterrows():
    if type(row['price']) == pd._libs.missing.NAType:
        units_dict = list(eval(row['units']))
        for x in units_dict:
            list2add = [[row['statusText'], x['price'], row['address'],
                         row['addressZipcode'], x['beds'], '0', '0',
                         row['rentZestimate'], row['variableData.text'],
                         row['units']]]
            df4append = df4append.append(pd.DataFrame(list2add,
                                          columns=full_df.columns),
                                          ignore_index=True)
        full_df.drop(index, inplace=True)
# print('append shape:')
# print(df4append.shape)

## clean and append fixed rows
cleaned_df = full_df.append(df4append, ignore_index=True)
cleaned_df['price'] = cleaned_df['price'].str.replace('+','')
cleaned_df['price'] = cleaned_df['price'].str.replace('$','')
cleaned_df['price'] = cleaned_df['price'].str.replace(',','')
cleaned_df['price'] = cleaned_df['price'].str.replace('/mo','')
# print('final shape:')
# print(cleaned_df.shape)

## save full edited and cleaned df
cleaned_df.to_csv('./Zillow_Rentals_Dec102022.csv')

In [ ]:
Full_dataset = pd.read_csv('./Zillow_Rentals_Dec102022.csv')

grouped = Full_dataset.groupby(Full_dataset.addressZipcode)
df_92037 = grouped.get_group(92037)
df_92109 = grouped.get_group(92109)
df_92110 = grouped.get_group(92110)
df_92117 = grouped.get_group(92117)
df_92121 = grouped.get_group(92121)
df_92122 = grouped.get_group(92122)
df_92126 = grouped.get_group(92126)




In [ ]:
print(df_92037.shape)
print(df_92109.shape)
print(df_92110.shape)
print(df_92117.shape)
print(df_92121.shape)
print(df_92122.shape)
print(df_92126.shape)